In [ ]:
import os
import time
import json
import pandas as pd
from tqdm import tqdm
from pycocotools.coco import COCO
 
def trans_id(category_id):
    names = []
    namesid = []
    for i in range(0, len(cats)):
        names.append(cats[i]['name'])
        namesid.append(cats[i]['id'])
    index = namesid.index(category_id)
    return index
    

anno = './anno/instances_val2017.json'
xml_dir = './xml/'
 
coco = COCO(anno) 
cats = coco.loadCats(coco.getCatIds())  
 
# Create anno dir
dttm = time.strftime("%Y%m%d%H%M%S", time.localtime())

with open(anno, 'r') as load_f:
    f = json.load(load_f)
 
imgs = f['images']  
 
cat = f['categories']
df_cate = pd.DataFrame(f['categories'])                     
df_cate_sort = df_cate.sort_values(["id"], ascending=True)  
categories = list(df_cate_sort['name'])                     
print('categories = ', categories)
df_anno = pd.DataFrame(f['annotations'])                    
 
for i in tqdm(range(len(imgs))): 
    xml_content = []
    file_name = imgs[i]['file_name']   
    height = imgs[i]['height']
    img_id = imgs[i]['id']
    width = imgs[i]['width']
    
    version =['"1.0"','"utf-8"'] 
 

    xml_content.append("<?xml version=" + version[0] +" "+ "encoding="+ version[1] + "?>")
    xml_content.append("<annotation>")
    xml_content.append("    <filename>" + file_name + "</filename>")
    xml_content.append("    <size>")
    xml_content.append("        <width>" + str(width) + "</width>")
    xml_content.append("        <height>" + str(height) + "</height>")
    xml_content.append("        <depth>"+ "3" + "</depth>")
    xml_content.append("    </size>")
 
  
    annos = df_anno[df_anno["image_id"].isin([img_id])]  
 
    for index, row in annos.iterrows():  
        bbox = row["bbox"]
        category_id = row["category_id"]
        cate_name = categories[trans_id(category_id)]
 
        # add new object
        xml_content.append("    <object>")
        xml_content.append("        <name>" + cate_name + "</name>")
        xml_content.append("        <truncated>0</truncated>")
        xml_content.append("        <difficult>0</difficult>")
        xml_content.append("        <bndbox>")
        xml_content.append("            <xmin>" + str(int(bbox[0])) + "</xmin>")
        xml_content.append("            <ymin>" + str(int(bbox[1])) + "</ymin>")
        xml_content.append("            <xmax>" + str(int(bbox[0] + bbox[2])) + "</xmax>")
        xml_content.append("            <ymax>" + str(int(bbox[1] + bbox[3])) + "</ymax>")
        xml_content.append("        </bndbox>")
        xml_content.append("    </object>")
    xml_content.append("</annotation>")
 
    x = xml_content
    xml_content = [x[i] for i in range(0, len(x)) if x[i] != "\n"]

    xml_path = os.path.join(xml_dir, file_name.split('j')[0]+'xml')
    print(xml_path)
    with open(xml_path, 'w+', encoding="utf8") as f:
        f.write('\n'.join(xml_content))
    xml_content[:] = []